In [1]:
import sklearn
sklearn.__version__
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.neural_network import MLPClassifier

In [2]:
data = pd.read_csv("weather_data.csv", delimiter = ";")
data.head()

,time,month,temperature,feelslike,wind,direction,gust,cloud,humidity,precipitation,pressure,weather
0,0:00,1,24.0,28.0,8.0,ENE,12.0,4.0,86.0,0.0,1012.0,Clear
1,3:00,1,23.0,27.0,8.0,NE,10.0,4.0,88.0,0.0,1011.0,Clear
2,6:00,1,23.0,26.0,8.0,NNE,11.0,7.0,85.0,0.0,1012.0,Sunny
3,9:00,1,28.0,33.0,11.0,NNE,13.0,6.0,64.0,0.0,1012.0,Sunny
4,12:00,1,31.0,35.0,10.0,ENE,12.0,62.0,53.0,0.0,1010.0,Partly cloudy


In [3]:
# Xếp loại các loại thời tiết thành 3 dạng là có mưa, mưa ít và không mưa
list_norain = ['Clear', 'Cloudy', 'Mist', 'Sunny', 'Partly cloudy', 'Thundery outbreaks possible']
list_ltrain = ['Light drizzle','Light rain','Light rain shower', 'Patchy light drizzle', 'Patchy light rain', 'Patchy light rain with thunder','Patchy rain possible']
list_rain = ['Heavy rain','Heavy rain at times','Moderate or heavy rain shower','Moderate rain', 'Moderate rain at times', 'Overcast','Torrential rain shower']

In [4]:
# Lấy ra cột y để train
# Cột y này là giá trị thời tiết được xếp loại
y = []
for w in data.weather:
    if w in list_norain:
        y+= ['norain']
    elif w in list_ltrain:
        y+= ['ltrain']
    elif w in list_rain:
        y+= ['rain']
    else:
        y+= ['unknown']
# Do input x có giá trị dự đoán là giá trị thời tiết của 3 giờ tiếp theo
# nên chúng ta xóa dòng đầu tiên
y.pop(0)

'norain'

In [5]:
# Do input x có giá trị dự đoán là giá trị thời tiết của 3 giờ tiếp theo nhưng không có giá trị đó nên xóa dòng cuối cùng
x = data.drop(8511)

In [6]:
# Tách tập train và tập test
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.1, stratify = y, random_state=0)

In [7]:
# Tách tập train và tập validation
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size = 0.2, stratify = y_train, random_state=0)

In [8]:
# Góm nhóm các giá trị của các cột có kiểu category
# Cột month gom thành 2 giá trị gồm "mua" (5->11) và "kho" (12->4)
# Cột direction thì xóa bỏ kí tự đầu tiên nếu vượt quá 2 ký tự
# Cột weather thì gom nhóm giống như khi gom nhóm y
class ColStd(BaseEstimator, TransformerMixin):
    def fit(self, X_df, y=None):
        return self
    def transform(self, x_df,y=None):
        df = x_df
        temp_month =[]
        for x in df['month']:
            if x in [5,6,7,8,9,10,11]:
                temp_month += ['mua'] 
            elif x in [12,1,2,3,4]:
                temp_month += ['kho']
            else:
                temp_month += ['kho']
        df.month = temp_month
        temp_dic = []
        for x in df['direction']:
            if len(x) == 3:
                temp_dic += [x[1:]]
            else:
                temp_dic += [x]
        df.direction = temp_dic
        temp_wea = []
        for w in df['weather']:
            if w in list_norain:
                temp_wea+= ['norain']
            elif w in list_ltrain:
                temp_wea+= ['ltrain']
            elif w in list_rain:
                temp_wea+= ['rain']
            else:
                temp_wea+= ['unknown']
        df.weather = temp_wea
        return df

In [9]:
# Tạo các Pipeline để tiền xử lý các giá trị có trong dữ liệu
nume_cols_zero = ['temperature', 'feelslike', 'wind', 'gust', 'cloud', 'humidity', 'pressure']
nume_cols_nan = ['temperature', 'feelslike', 'wind', 'gust', 'cloud', 'humidity','precipitation', 'pressure']
cate_cols = ['time', 'month', 'direction', 'weather']

nume_trans_zero = Pipeline(steps=[
    ('imputer', SimpleImputer(missing_values = 0, strategy='mean'))
])
nume_trans_nan = Pipeline(steps=[
    ('imputer', SimpleImputer(missing_values = np.nan, strategy = 'mean'))
])
cate_trans = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy= 'most_frequent')),
    ('onehot', OneHotEncoder(dtype=int, handle_unknown= 'ignore'))
])

cols_trans = ColumnTransformer(
    transformers=[
        ('nume_nan', nume_trans_nan, nume_cols_nan),
        ('nume_zero',nume_trans_zero, nume_cols_zero),
        ('cate', cate_trans, cate_cols)
])

preprocess_pipeline = Pipeline(steps=[
    ('colstd', ColStd()),
    ('coltrans', cols_trans),
    ('std', StandardScaler())
])

# Tiền xử lý tập train x
preprocessed_train_X = preprocess_pipeline.fit_transform(x_train)

In [10]:
# Tiền xử lý tập validation x
preprocess_val_X = preprocess_pipeline.transform(x_train)

In [11]:
# Tạo ra full_pipeline bao gồm tiền xử lý và model
full_pipeline = Pipeline(steps=[
    ('pre', preprocess_pipeline),
    ('mlpclf', MLPClassifier(activation='tanh', solver='lbfgs', random_state=0, max_iter=500))
])
#hidden_layer_sizes = (10, 20, 50, 100)

In [20]:
# train và tìm ra các tham số tốt nhất
train_errs = []
val_errs = []
alphas = [0.1, 1, 10, 100, 1000]
hidden_layer_sizes = [10, 50, 100]
best_val_err = float('inf'); best_alpha = None; best_hls = None;
for alpha in alphas:
    for hls in hidden_layer_sizes:
        print(f"{alpha}, {hls}")
        full_pipeline.set_params(mlpclf__alpha=alpha, mlpclf__hidden_layer_sizes=hls)
        full_pipeline.fit(x_train,y_train)
        train_err =100*(1 - full_pipeline.score(x_train,y_train))
        val_err =100*(1 - full_pipeline.score(x_val,y_val))
        train_errs += [train_err]
        val_errs +=[val_err]
        if ((val_err < best_val_err)):
            best_val_err = val_err
            best_alpha = alpha
            best_hls = hls


0.1, 10
0.1, 50
0.1, 100
1, 10
1, 50
1, 100
10, 10
10, 50
10, 100
100, 10
100, 50
100, 100
1000, 10
1000, 50
1000, 100


In [21]:
print(best_hls)

50


In [14]:
# Nối tập train và val lại, sau đó train model cuối cùng
X = pd.concat((x_train, x_val), axis=0)
Y = np.concatenate((y_train, y_val), axis=0)
full_pipeline.set_params(mlpclf__alpha=10)
full_pipeline.fit(X, Y)

Pipeline(memory=None,
         steps=[('pre',
                 Pipeline(memory=None,
                          steps=[('colstd', ColStd()),
                                 ('coltrans',
                                  ColumnTransformer(n_jobs=None,
                                                    remainder='drop',
                                                    sparse_threshold=0.3,
                                                    transformer_weights=None,
                                                    transformers=[('nume_nan',
                                                                   Pipeline(memory=None,
                                                                            steps=[('imputer',
                                                                                    SimpleImputer(add_indicator=False,
                                                                                                  copy=True,
                                    

In [15]:
full_pipeline.score(x_test, y_test)

C:\Users\Dell\Anaconda3\lib\site-packages\pandas\core\generic.py:5096: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


0.8169014084507042